In [1]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import re
import chardet
import pandas as pd
import time
import os
from db_conn.connection import *
from IPython.display import clear_output

In [197]:
rows = []
files = [x for x in os.listdir('./html_source/subchapters') if x != '.DS_Store']
cnt = 0
total= len(files)
for x in files:
    
    clear_output(wait=True)
    cnt += 1
    print(f'file: {cnt} out of {total}....{x}')

    with open(f'./html_source/subchapters/{x}', 'rb') as f:
        soup = BeautifulSoup(f)

    _title = soup.find('title').text.strip() if soup.find('title') else None

    _heading= None
    _body = None

    for y in soup.find_all('p'):
        
        if 'heading' in y.get('class')[0].lower():
            _heading = y.text.strip()
            _body = None

        if 'body' in y.get('class')[0].lower():
            _body = y.text

        row = pd.DataFrame({
                'file_name':    [x],
                'title':        [_title],
                'heading':      [_heading],
                'body_text':    [_body]
            })

        rows.append(row)



file: 1493 out of 1493....03FAH012110.html


In [199]:
df = pd.concat(rows, ignore_index=True)
df.to_sql('fam', engine(), if_exists='replace', index=False)

833

In [204]:
df_g = (
    df.loc[df['body_text'].notna(), :]
    .reset_index(drop=True)
    .groupby(['file_name', 'title', 'heading'], as_index = False)['body_text']
    .apply(lambda x: ' \n '.join(x))
)

In [205]:
df_g

,file_name,title,heading,body_text
0,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",,\n\n\n\n\n \n \n\n\n\n\n \n \n\n\n\n\n \n \n\n...
1,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",1 FAM 011\n \n AUTHORITY FOR CONDUCTING ...,\n The Constitution vests in the President ...
2,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",1 FAM 011.1\n \n Scope,\n a. The functional statements or organiza...
3,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",1 FAM 011.2\n \n Role of Department of S...,\n The Department of State exists to assist...
4,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",1 FAM 012\n \n THE SECRETARY OF STATE’S ...,\n a. The Secretary of State’s basic author...
...,...,...,...,...
20641,20FAM170101.html,20 FAM 1701.1 SCOPE AND BUSINESS DRIVERS (DATA...,20 FAM 1701.1-2\n \n Business Drivers\n(...,\n Reserved.\n \n \n Reserved.\n
20642,20FAM170102.html,20 FAM 1701.2 ESSENTIAL CONCEPTS (DATA AND AI ...,20 FAM 1701.2-1\n \n Introduction,\n Reserved.\n \n \n \n \n
20643,20FAM170201.html,20 FAM 1702.1 ROLES AND RESPONSIBILITIES (DATA...,20 FAM 1702.1-1\n \n Introduction,\n Reserved.\n \n \n Reserved.\n
20644,20FAM170202.html,20 FAM 1702.2 COMPLIANCE STANDARDS (DATA AND A...,20 FAM 1702.2-1\n \n Introduction,\n Reserved.\n \n \n Reserved.\n


In [206]:
df_g['heading'] = df_g['heading'].str.split('\n')
df_g

,file_name,title,heading,body_text
0,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",[],\n\n\n\n\n \n \n\n\n\n\n \n \n\n\n\n\n \n \n\n...
1,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...","[1 FAM 011, , AUTHORITY FOR CONDUCTING...",\n The Constitution vests in the President ...
2,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...","[1 FAM 011.1, , Scope]",\n a. The functional statements or organiza...
3,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...","[1 FAM 011.2, , Role of Department of ...",\n The Department of State exists to assist...
4,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...","[1 FAM 012, , THE SECRETARY OF STATE’S...",\n a. The Secretary of State’s basic author...
...,...,...,...,...
20641,20FAM170101.html,20 FAM 1701.1 SCOPE AND BUSINESS DRIVERS (DATA...,"[20 FAM 1701.1-2, , Business Drivers, ...",\n Reserved.\n \n \n Reserved.\n
20642,20FAM170102.html,20 FAM 1701.2 ESSENTIAL CONCEPTS (DATA AND AI ...,"[20 FAM 1701.2-1, , Introduction]",\n Reserved.\n \n \n \n \n
20643,20FAM170201.html,20 FAM 1702.1 ROLES AND RESPONSIBILITIES (DATA...,"[20 FAM 1702.1-1, , Introduction]",\n Reserved.\n \n \n Reserved.\n
20644,20FAM170202.html,20 FAM 1702.2 COMPLIANCE STANDARDS (DATA AND A...,"[20 FAM 1702.2-1, , Introduction]",\n Reserved.\n \n \n Reserved.\n


In [207]:
df_g['ref'] = df_g['heading'].apply(lambda x: ''.join([i for i in x if 'fam' in i.lower()]) if x else None)
df_g['heading'] = df_g['heading'].apply(lambda x: ''.join([i for i in x if 'fam' not in i.lower()]) if x else None)

In [209]:
df_g

,file_name,title,heading,body_text,ref
0,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",,\n\n\n\n\n \n \n\n\n\n\n \n \n\n\n\n\n \n \n\n...,
1,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",AUTHORITY FOR CONDUCTING FOREIGN RELAT...,\n The Constitution vests in the President ...,1 FAM 011
2,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",Scope,\n a. The functional statements or organiza...,1 FAM 011.1
3,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",Role of Department of State,\n The Department of State exists to assist...,1 FAM 011.2
4,01FAM0010.html,"1 FAM 010 AUTHORITY, RESPONSIBILITY, AND ORGAN...",THE SECRETARY OF STATE’S AUTHORITY,\n a. The Secretary of State’s basic author...,1 FAM 012
...,...,...,...,...,...
20641,20FAM170101.html,20 FAM 1701.1 SCOPE AND BUSINESS DRIVERS (DATA...,Business Drivers(DATA AND AI MANAGEMEN...,\n Reserved.\n \n \n Reserved.\n,20 FAM 1701.1-2
20642,20FAM170102.html,20 FAM 1701.2 ESSENTIAL CONCEPTS (DATA AND AI ...,Introduction,\n Reserved.\n \n \n \n \n,20 FAM 1701.2-1
20643,20FAM170201.html,20 FAM 1702.1 ROLES AND RESPONSIBILITIES (DATA...,Introduction,\n Reserved.\n \n \n Reserved.\n,20 FAM 1702.1-1
20644,20FAM170202.html,20 FAM 1702.2 COMPLIANCE STANDARDS (DATA AND A...,Introduction,\n Reserved.\n \n \n Reserved.\n,20 FAM 1702.2-1


In [210]:
df_g.to_sql('fam_grouped', engine(), if_exists='replace', index=False)

646